# 🏛️ IstanBuilders - Turkish Complaint Classification System

## 📦 SECTION 1: SETUP & DEPENDENCIES
Install required packages and import libraries.

In [1]:
# Cell 1: Dependencies are pre-installed in the virtual environment
# If you need to reinstall, run: pip install -r requirements.txt
# All dependencies are already available in the istanbuilders environment
print("✓ Dependencies are already installed in the virtual environment")
print("  Environment: .venv (Python 3.12)")
print("  To reinstall: pip install -r requirements.txt")

✓ Dependencies are already installed in the virtual environment
  Environment: .venv (Python 3.12)
  To reinstall: pip install -r requirements.txt


In [2]:
# Cell 2: Imports - Basic libraries first
# Fix OpenMP conflict (must be before numpy/torch imports)
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import json
import pickle
import numpy as np
import pandas as pd
import re
from typing import Dict, List, Tuple
from datetime import datetime, timedelta
import uuid
import random

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("✓ Basic imports loaded successfully!")

✓ Basic imports loaded successfully!


In [3]:
# Cell 2b: ML/NLP imports (these can be memory-intensive)
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

print("✓ ML imports loaded successfully!")

c:\Users\islek\Desktop\ibb-akilli-sehir-hackathon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ ML imports loaded successfully!


In [4]:
# Cell 2c: Database imports
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

print("✓ Database imports loaded successfully!")

✓ Database imports loaded successfully!


---
## 🤖 SECTION 2: MODEL & TEMPLATES
Load Turkish BERT model and define category templates for classification.

In [5]:
# Cell 3: Define FINAL templates
TEMPLATES = {
    'su_kanalizasyon': [
        "İSKİ ekipleri 3 gündür gelmiyor, mahallede su kesintisi devam ediyor",
        "Su sayacında sızıntı var, fatura çok yüksek geldi, İSKİ kontrol etmeli",
        "Kanalizasyon kapağı çökmüş, sokakta tehlikeli, çocuklar düşebilir acil",
        "Musluktan su akmıyor, 2 gündür susuzuz, İSKİ'yi aradık cevap yok",
        "Su borusu patladı, cadde göl oldu, İSKİ ekipleri acil müdahale gerekli",
        "Lağım suyu sokağa taşıyor, koku dayanılmaz, kanalizasyon sağlık sorunu",
        "Kanalizasyon tıkanmış, bodrum kata su basıyor, İSKİ pompaj gerekli",
        "İçme suyu bulanık ve kokulu, içilebilir mi bilmiyoruz, İSKİ kontrol",
        "Apartman önünde su sızıntısı mevcut, zemin kayıyor, tehlikeli durum",
        "Atık su kanalı taşıyor, sokak pis durumda, kanalizasyon bakteri riski",
        "Su kesintisi bildirimi yapılmadı, iki gündür susuzuz, İSKİ bilgi yok",
        "Kanalizasyon kokusu dayanılmaz, pencere açamıyoruz, İSKİ müdahale",
    ],

    'atik_yonetimi_temizlik': [
        # Atık yönetimi templates
        "Çöp konteynerleri 4 gündür dolu, sokakta koku oluştu, çöp toplanmıyor",
        "Geri dönüşüm kumbaraları hiç boşaltılmıyor, yanına çöp atılıyor artık",
        "Çöp arabası gelmiyor, atıklar sokakta birikiyor, hijyen sorunu oluştu",
        "Konteynerlerin kapağı kırık, çöpler etrafa saçılıyor, kediler karıştırıyor",
        "Moloz ve büyük eşyalar sokağa bırakılmış, toplanmıyor, kaldırımı tıkıyor",
        "Organik atık konteyneri yok mahallemizde, kompost talebi yapıyoruz",
        "Çöp toplama saatleri çok erken, sabah beşte gürültü var, uyuyamıyoruz",
        "Tıbbi atık kutuları sokağa konulmuş, enfeksiyon riski var, tehlikeli",
        "Atık toplama düzensiz, bazen geliyor bazen gelmiyor, program yapılmalı",
        "Çöp konteynerleri yetersiz, sürekli taşıyor, daha fazla konteyner lazım",
        "Geri dönüşüm kutuları dolu ama boşaltılmıyor, sistematik sorun var",
        "Büyük eşya atık toplama günü ne zaman, dolap atıyoruz bilgi lazım",
        # Temizlik templates
        "Sokak süpürülmüyor, yapraklar her yerde, temizlik ekibi gelmiyor",
        "Cadde üzerinde çamur birikmiş, yağmur sonrası temizlik yapılmadı",
        "Kaldırımlar kirli, pislik birikmiş, temizlik araçları göremedik",
        "Duvar yazıları temizlenmiyor, grafiti kaldırılmalı, görüntü kirliliği",
        "Ağaç yaprakları sokağı kaplamış, süpürge aracı gelmiyor hiç",
        "Kış sonrası tuz kalıntıları kaldırımda, temizlik yapılmıyor",
        "Pazarcı kalkıyor ama sokak temizlenmiyor, pislik kalıyor",
        "Kuş pisliği bankları kaplamış, temizlik ekibi gelmiyor düzenli",
        "Sokakta çamur birikintisi var, yıkama aracı gönderin lütfen",
        "Duraklar kirli, temizlik yapılmıyor, banklarda leke var",
        "Kaldırımlar yağ lekeli, restoran önü temizlenmiyor hiç",
        "Kar temizleme yapılmadı, kaldırımlar buzlanmış, kayma riski",
    ],

    'ulasim_trafik': [
        "İETT otobüsü çok geç kalıyor, işe geç kaldık, sefer artırılmalı",
        "Otobüs saatlerce gelmiyor, İETT mobil uygulaması yanlış bilgi veriyor",
        "Metrobüs durağında rampa yok, tekerlekli sandalye kullanamıyoruz",
        "Tramvay seferleri çok seyrek, sabah işe geç kalıyoruz, yetersiz",
        "Metro arızalandı, yolcular peronlarda bekliyor, İETT bilgi vermiyor",
        "Akbil dolum noktası arızalı, üç gündür İstanbul Kart dolduramıyoruz",
        "Toplu taşıma kartı okutmuyor, bakiye var ama İETT sistemi tanımıyor",
        "Otobüs şoförü kaba davrandı, yaşlı yolcuya bağırdı, İETT şikayet",
        "UKOME kararıyla yol kapatıldı, alternatif rota yok, trafik felç oldu",
        "Trafik lambası çalışmıyor, kavşakta kaza riski var, acil tamir gerekli",
        "Durak tabelası yanlış bilgi veriyor, hangi hatlar geçiyor belli değil",
        "Yeni açılan metro hattı İETT haritasında görünmüyor, güncelleme lazım",
        "Otopark yeri çok az, rezidans önü araç park yasağı var mı",
        "Araç park yeri çizgileri silinmiş, nereye park edeceğiz belli değil",
        "Cadde üstü otopark çizgileri yok, araç park ederken zorlanıyoruz",
    ],

    'yol_altyapi': [
        "Cadde üzerinde dev çukur var, araba lastiği patladı, yol tamiri lazım",
        "Kaldırımlar tamamen çökmüş, yayalar yolda yürümek zorunda kalıyor",
        "Yol çalışması iki aydır devam ediyor, ilerleme yok, ne zaman bitecek",
        "Asfalt kalitesi çok kötü, yağmurda su birikintileri oluşuyor, tehlikeli",
        "Yaya geçidi çizgileri silinmiş, göremiyoruz, trafik kazası riski var",
        "Bisiklet yolu bozuk, düşme riski var, yol tamir edilmesi şart",
        "Bordür taşları kopmuş, yaşlılar tökezliyor, kaldırım kırık kemik riski",
        "Yol kenarı çökmüş, ağaç kökleri asfaltı delmis, tehlike oluşturmuş",
        "Kavşakta yol çalışması var ama işaretleme yok, kaza oluyor",
        "Altyapı kazısı bitmiş ama yol açılmamış, açık çukur tehlikeli",
        "Yol çukurları çok fazla, araç süspansiyonu bozuluyor, şikayet var",
        "Asfalt tamiri yapıldı ama yama kalitesiz, tekrar bozuldu",
    ],

    'yesil_alan_bahce': [
        "Yeşil alanda ağaçlar kurumuş, sulama yapılmıyor, bakım gerekli acil",
        "Çocuk oyun alanındaki salıncaklar kırık, tehlikeli, yaralanma riski var",
        "Yeşil alana inşaat yapılıyor, mahalle bahçemiz gidiyor, engellenmeli",
        "Bahçe bankları çürümüş, oturulamıyor, yeşil alan mobilyası yenilenmeli",
        "Spor aletleri paslanmış, açık hava fitness alanı kullanılamaz durumda",
        "Çocuk oyun alanında cam kırıkları var, tehlikeli, temizlik gerekli",
        "Ağaç budama yapılmıyor, dallar elektrik tellerine değiyor, tehlikeli",
        "Yeşil alanda yabani otlar her yeri kaplamış, bahçe bakımsızlık had safhada",
        "Mahallemizde yeşil alan yetersiz, çocukların oynayacağı bahçe yok",
        "Oyun alanı ekipmanları kırık, tamir edilmiyor, güvenlik eksikliği",
        "Piknik masaları çürümüş, bahçe mobilyaları değiştirilmeli acil",
        "Koşu parkuru zemini bozuk, yeşil alan yürüyüş yolu yenilensin",
    ],

    'aydinlatma': [
        "Sokak lambası üç haftadır yanmıyor, gece karanlıkta kalıyoruz güvenlik",
        "Cadde aydınlatması yetersiz, akşamları göremiyoruz, kaza riski var",
        "Mahalle içi sokak aydınlatması yok, akşam karanlık, girilemiyor",
        "Lamba direkleri var ama ampuller yanmıyor, değiştirilmesi lazım acil",
        "Elektrik direğindeki kablolar dışarı çıkmış, çarpılma riski var",
        "Sokakta hiç lamba yok, beş yüz metre karanlık, direk dikilmeli",
        "LED lambalara geçildi ama çok parlak, uykularımız kaçıyor rahatsız",
        "Sokak lambası gün boyu yanıyor, elektrik israfı, sensör arızalı",
        "Yeşil alan yakınında aydınlatma yok, akşam yürüyemiyoruz, lamba lazım",
        "Sokak armatürleri kırık, cam parçaları yere düşüyor, tehlike var",
        "Cadde lambaları gündüz de yanıyor, elektrik israfı, sensör bozuk",
        "Mahallede aydınlatma yokken akşamları güvenlik sorunu yaşıyoruz",
    ],

    'sosyal_yardim': [
        "65+ İstanbul Kart başvurum iki aydır beklemede, yaş belgesi verdim",
        "Engelli kartı talebi reddedildi, sağlık raporu yenilenmeli mi soruyorum",
        "Gıda kolisi yardımı için başvurdum, muhtardan onay gerekiyor mu",
        "Sedep kartı hangi marketlerde geçerli, kullanım alanları neler bilgi",
        "Kömür yardımı talebi gönderdim, kış ayında verilecek mi bekliyorum",
        "Sosyal destek ödemesi hesabıma geçmedi, ne zaman yatırılacak bilgi",
        "Mavi kart başvurusu onaylanmadı, gelir tespiti nasıl yapılıyor",
        "İhtiyaç sahibi komşum var, gıda yardımı programı nasıl başvurulur",
        "Eğitim bursu başvurum reddedildi, hangi şartlar gerekiyor bilgi",
        "Yaşlılık aylığı başvurum onaylanmadı, gelir şartı nedir soruyorum",
        "Evde bakım hizmeti için başvurdum, süreç nasıl ilerliyor bilgi",
        "Nakdi yardım ödemesi gecikti, hesaba ne zaman yatacak soruyorum",
    ],

    'fatura_odeme': [
        "Emlak vergisi faturası yanlış hesaplanmış, metrekare hatası var itiraz",
        "Su faturası normalden üç kat yüksek geldi, sızıntı yok hata var",
        "Online ödeme yaptım ama sistem kaydetmemiş, param nerede düzeltilsin",
        "Çevre temizlik vergisi çok yüksek, komşularla farklı, yanlış hesap",
        "Borç yok ama tahsilat mektubu geldi, kontrol edilmeli acil",
        "Ödeme planı yapmak istiyorum, taksit mümkün mü, ödeyemiyorum",
        "Fatura tahakkuk hatası var, ödedim ama tekrar borç çıkmış",
        "İlan reklam vergisi kesilmiş ama tabela yok işyerimde, hatalı",
        "Otomatik ödeme talimatı verdim ama çekilmemiş, gecikme faizi eklenmiş",
        "Fatura iptal ettirdim ama tekrar geldi, düzeltme yapılmamış",
        "Borç itirazı yapmak istiyorum, nasıl yapabilirim bilgi almak",
        "Emlak vergisi çok yüksek geldi, ödeme zorlanıyorum, taksit lazım",
    ],

    'ruhsat_imar': [
        "Yapı ruhsatı başvurusu yaptım, üç aydır sonuç yok, ne zaman onay",
        "İmar durumu belgesi aldım ama bilgiler hatalı, düzeltilebilir mi",
        "İkamet belgesi için başvurdum, eksik evrak dendi, hangi belgeler",
        "İşyeri açma ruhsatı için randevu alamıyorum, sistem dolu gösteriyor",
        "Evlenme başvurusu için gerekli belgeler neler, bilgi almak istiyorum",
        "Belge tasdik işlemi için gittim, noter onaylı olmalıymış dendi",
        "Dilekçeme iki aydır cevap gelmedi, başvuru durumu nasıl öğrenirim",
        "Evrak teslim ettim ama sistem kayıt göstermiyor, tekrar vermeli",
        "Ruhsat yenileme başvurusu yaptım, eski ruhsat geçerli mi süre",
        "Otopark projesi için başvurdum, hangi evraklar eksik bilgi",
        "İmar durumu öğrenmek istiyorum, nasıl başvuru yapılır bilgi",
        "Yapı ruhsatı işlemleri çok uzun sürüyor, hızlandırılmalı acil",
        "Binaya ruhsatsız kat çıkıldı, imar planına aykırı, denetim yapın lütfen",
        "Ruhsatsız kat ilavesi yapılıyor, izinsiz inşaat var, kontrol edin",
        "Komşu binaya izinsiz kat eklendi, hem ışık kesti hem tehlikeli",
        "İmar planına aykırı kat çıkılıyor, ruhsatsız yapı var, denetim gerekli",
        "İnşaatta izinsiz çalışma var, denetim gelmiyor mu, projeye uymuyor",
        "Ruhsatsız inşaat yapılıyor, izinsiz çalışma devam ediyor, kontrol edin",
        "İzinsiz yapı çalışması var, projeye uymayan şeyler yapıyorlar",
        "Denetimsiz inşaat sürüyor, ruhsatsız çalışma yapılıyor, müdahale edin",
    ],

    'sikayet_takip': [
        "ALO 153'ü arıyorum, yirmi dakikadır bekletiliyor, hiç bağlanmıyorum",
        "Çağrı merkezi görevlisi ilgisizdi, soruları cevaplamadı, yardım yok",
        "Şikayetimi ilettim ama takip numarası verilmedi, nasıl takip ederim",
        "153'e beş kere aradım, hep farklı bilgi verdiler, hangisi doğru",
        "Geri arama yapılacak denmişti, bir haftadır kimse aramadı bekliyorum",
        "Şikayet kaydı yapıldı ama sistem numarası göstermiyor, kayboldu mu",
        "Çağrı merkezi hatları sürekli meşgul, acil durumda ulaşamıyoruz",
        "Yanlış departmana yönlendirildim, üç kere aktarıldım, zaman kaybı",
        "Otomatik telefon sistemi bozuk, tuşlara basıyorum ama algılamıyor",
        "Görevli telefonu kapattı, şikayetimi dinlemedi, saygısızlık var",
        "Beyaz masa'dan geri dönüş sözü aldım, kimse aramadı takip yok",
        "Müşteri hizmetleri cevap vermiyor, telefon bekleme süresi çok uzun",
    ],

    'dijital_sistem': [
        "İBB web sitesi Chrome tarayıcısında açılmıyor, hata mesajı veriyor",
        "İstanbul Senin uygulaması sürekli donuyor, güncelleme yükledim ama",
        "E-belediye sistemine giriş yapamıyorum, TC kimlik doğrulama hatası",
        "QR kod okutma çalışmıyor, mobil uygulama kamera izni istiyor",
        "SMS doğrulama kodu gelmiyor, telefon numarası doğru kayıtlı ama",
        "PDF belge indirme butonu tıklanmıyor, 404 hatası alıyorum sayfa",
        "Mobil uygulamada harita görünmüyor, konum servisi açık ama",
        "Sistem bakımda diyor ama üç gündür açılmıyor, ne zaman düzelir",
        "Online başvuru formunda alanlar açılmıyor, teknik sorun var",
        "Şifre sıfırlama linki çalışmıyor, e-posta geldi ama tıklanmıyor",
        "Dosya yükleme çalışmıyor, PDF kabul etmiyor, başka format denemeliyim",
        "Site Safari'de açılmıyor, Chrome'da sorun var, düzeltilmeli",
    ],
}

# Print summary
total_templates = sum(len(v) for v in TEMPLATES.values())
avg_length_per_template = sum(len(t.split()) for templates in TEMPLATES.values() for t in templates) / total_templates

print(f"Statistics:")
print(f"  Categories:         {len(TEMPLATES)}")
print(f"  Total templates:    {total_templates}")
print(f"  Avg words/template: {avg_length_per_template:.1f}")
print(f"  Templates/category: {total_templates/len(TEMPLATES):.1f}")

print("\nAll 11 Categories:")
for i, cat in enumerate(TEMPLATES.keys(), 1):
    print(f"  {i:2d}. {cat}")

Statistics:
  Categories:         11
  Total templates:    155
  Avg words/template: 8.8
  Templates/category: 14.1

All 11 Categories:
   1. su_kanalizasyon
   2. atik_yonetimi_temizlik
   3. ulasim_trafik
   4. yol_altyapi
   5. yesil_alan_bahce
   6. aydinlatma
   7. sosyal_yardim
   8. fatura_odeme
   9. ruhsat_imar
  10. sikayet_takip
  11. dijital_sistem


In [6]:
# Cell 4: Load sentence transformer model
model = SentenceTransformer('emrecan/bert-base-turkish-cased-mean-nli-stsb-tr')
print("Model loaded successfully!")

Model loaded successfully!


In [7]:
# Cell 5: Build embeddings

# Flatten all templates with category labels
all_templates = []
template_categories = []

for category, templates in TEMPLATES.items():
    for template in templates:
        all_templates.append(template)
        template_categories.append(category)

# Generate embeddings
template_embeddings = model.encode(all_templates, show_progress_bar=True)

print(f"Created embeddings for {len(all_templates)} templates")
print(f"Embedding shape: {template_embeddings.shape}")

Batches: 100%|██████████| 5/5 [00:24<00:00,  4.81s/it]

Created embeddings for 155 templates
Embedding shape: (155, 768)


In [8]:
# Cell 6: Define classification function (SEMANTIC ONLY)
def classify_complaint(complaint_text, model, template_embeddings, top_k=3):
    """
    Classify complaint using SEMANTIC SEARCH ONLY

    Args:
        complaint_text: The complaint to classify
        model: Sentence transformer model
        template_embeddings: Pre-computed template embeddings
        top_k: Number of top predictions to return

    Returns:
        dict with category, confidence, and top_k predictions
    """
    # Get complaint embedding
    complaint_embedding = model.encode([complaint_text])[0]

    # Calculate cosine similarities
    similarities = cosine_similarity(
        [complaint_embedding],
        template_embeddings
    )[0]

    # Get top matches
    top_indices = np.argsort(similarities)[::-1][:top_k]

    # Aggregate scores by category
    category_scores = {}
    for idx in range(len(similarities)):
        category = template_categories[idx]
        score = similarities[idx]

        if category not in category_scores:
            category_scores[category] = []
        category_scores[category].append(score)

    # Average scores per category
    category_avg_scores = {
        cat: np.mean(scores)
        for cat, scores in category_scores.items()
    }

    # Get top category
    top_category = max(category_avg_scores.items(), key=lambda x: x[1])

    # Get top k categories
    sorted_categories = sorted(
        category_avg_scores.items(),
        key=lambda x: x[1],
        reverse=True
    )[:top_k]

    return {
        'category': top_category[0],
        'confidence': float(top_category[1]),
        'top_k': [
            {'category': cat, 'score': float(score)}
            for cat, score in sorted_categories
        ]
    }


---
## 📂 SECTION 3: DATA LOADING & CLEANING
Load complaints and departments data, then remove duplicate entries.

In [9]:
complaints = pd.read_csv("data/complaints.csv")
complaints

,ticket_id,timestamp,complaint_text,department_id,district,neighborhood,latitude,longitude,platform
0,1,2024-12-25 22:13:26,"Günaydın, apartmanımızda 3 gündür su basıncı ç...",1,Büyükçekmece,Güzelce,41.029684,28.591755,İstanbul Senin
1,2,2024-12-25 08:35:29,"Merhabalar, Kennedy Sokağında sabahtan beri su...",1,Üsküdar,Beylerbeyi,41.043009,29.042758,İstanbul Senin
2,3,2024-12-26 22:18:04,"Günaydın, apartmanımızda 3 gündür su basıncı ç...",1,Beşiktaş,Levent,41.075523,29.010404,Email
3,4,2024-12-27 20:23:29,"İyi günler, su faturamız anormal derecede yüks...",1,Şişli,Mecidiyeköy,41.063021,28.991485,Email
4,5,2024-12-25 11:43:20,"Merhaba, sokağımızda 2 gündür kanalizasyon taş...",1,Beşiktaş,Levent,41.074852,29.010375,İstanbul Senin
...,...,...,...,...,...,...,...,...,...
995,996,2024-12-25 21:09:37,"Merhaba, uygulamada randevu alma özelliği yok ...",11,Bahçelievler,Bahçelievler,41.004971,28.845924,ALO153
996,997,2024-12-25 10:06:07,"Günaydın, mobil uygulamada fatura ödeme çalışm...",11,Çekmeköy,Alemdağ,41.045510,29.211545,ALO153
997,998,2024-12-27 07:23:35,"Selam, şifre sıfırlama maili 12 saattir gelmiy...",11,Avcılar,Firüzköy,40.989380,28.719322,ALO153
998,999,2024-12-27 19:42:29,"Selam, şifre sıfırlama maili 12 saattir gelmiy...",11,Kağıthane,Gültepe,41.074775,28.972791,ALO153


In [10]:
status_options = ["basvuru_alindi", "basvuru_islemde", "basvuru_cozumlendi"]
complaints['status'] = np.random.choice(status_options, size=len(complaints))
display(complaints.head())

,ticket_id,timestamp,complaint_text,department_id,district,neighborhood,latitude,longitude,platform,status
0,1,2024-12-25 22:13:26,"Günaydın, apartmanımızda 3 gündür su basıncı ç...",1,Büyükçekmece,Güzelce,41.029684,28.591755,İstanbul Senin,basvuru_alindi
1,2,2024-12-25 08:35:29,"Merhabalar, Kennedy Sokağında sabahtan beri su...",1,Üsküdar,Beylerbeyi,41.043009,29.042758,İstanbul Senin,basvuru_alindi
2,3,2024-12-26 22:18:04,"Günaydın, apartmanımızda 3 gündür su basıncı ç...",1,Beşiktaş,Levent,41.075523,29.010404,Email,basvuru_alindi
3,4,2024-12-27 20:23:29,"İyi günler, su faturamız anormal derecede yüks...",1,Şişli,Mecidiyeköy,41.063021,28.991485,Email,basvuru_alindi
4,5,2024-12-25 11:43:20,"Merhaba, sokağımızda 2 gündür kanalizasyon taş...",1,Beşiktaş,Levent,41.074852,29.010375,İstanbul Senin,basvuru_islemde


In [11]:
complaints

,ticket_id,timestamp,complaint_text,department_id,district,neighborhood,latitude,longitude,platform,status
0,1,2024-12-25 22:13:26,"Günaydın, apartmanımızda 3 gündür su basıncı ç...",1,Büyükçekmece,Güzelce,41.029684,28.591755,İstanbul Senin,basvuru_alindi
1,2,2024-12-25 08:35:29,"Merhabalar, Kennedy Sokağında sabahtan beri su...",1,Üsküdar,Beylerbeyi,41.043009,29.042758,İstanbul Senin,basvuru_alindi
2,3,2024-12-26 22:18:04,"Günaydın, apartmanımızda 3 gündür su basıncı ç...",1,Beşiktaş,Levent,41.075523,29.010404,Email,basvuru_alindi
3,4,2024-12-27 20:23:29,"İyi günler, su faturamız anormal derecede yüks...",1,Şişli,Mecidiyeköy,41.063021,28.991485,Email,basvuru_alindi
4,5,2024-12-25 11:43:20,"Merhaba, sokağımızda 2 gündür kanalizasyon taş...",1,Beşiktaş,Levent,41.074852,29.010375,İstanbul Senin,basvuru_islemde
...,...,...,...,...,...,...,...,...,...,...
995,996,2024-12-25 21:09:37,"Merhaba, uygulamada randevu alma özelliği yok ...",11,Bahçelievler,Bahçelievler,41.004971,28.845924,ALO153,basvuru_alindi
996,997,2024-12-25 10:06:07,"Günaydın, mobil uygulamada fatura ödeme çalışm...",11,Çekmeköy,Alemdağ,41.045510,29.211545,ALO153,basvuru_alindi
997,998,2024-12-27 07:23:35,"Selam, şifre sıfırlama maili 12 saattir gelmiy...",11,Avcılar,Firüzköy,40.989380,28.719322,ALO153,basvuru_alindi
998,999,2024-12-27 19:42:29,"Selam, şifre sıfırlama maili 12 saattir gelmiy...",11,Kağıthane,Gültepe,41.074775,28.972791,ALO153,basvuru_islemde


In [12]:
departments = pd.read_csv("data/departments.csv", sep=";")
departments

,department_id,department_name,category_name
0,1,İSKİ,su_kanalizasyon
1,2,Çevre Koruma ve Kontrol Dairesi,atik_yonetimi_temizlik
2,3,İETT,ulasim_trafik
3,4,Yol Yapım Dairesi,yol_altyapi
4,5,Park ve Bahçeler Dairesi,yesil_alan_bahce
5,6,Yol Bakım Dairesi,aydinlatma
6,7,Sosyal Hizmetler Dairesi,sosyal_yardim
7,8,Mali Hizmetler Dairesi,fatura_odeme
8,9,İmar Dairesi,ruhsat_imar
9,10,Beyaz Masa,sikayet_takip


In [13]:
# ====================
# MERGE COMPLAINTS WITH DEPARTMENTS - Get actual category names
# ====================

print("="*80)
print("MERGING COMPLAINTS WITH DEPARTMENTS")
print("="*80)

# Merge to get actual category names
complaints = complaints.merge(
    departments[['department_id', 'category_name']], 
    on='department_id', 
    how='left'
)

# Rename for clarity
complaints.rename(columns={'category_name': 'actual_category'}, inplace=True)

print(f"\n✓ Merged successfully")
print(f"  Total complaints: {len(complaints)}")
print(f"  Columns: {list(complaints.columns)}")
print(f"\n✓ Sample data:")
print(complaints[['ticket_id', 'department_id', 'actual_category', 'complaint_text']].head())
print("="*80)

MERGING COMPLAINTS WITH DEPARTMENTS

✓ Merged successfully
  Total complaints: 1000
  Columns: ['ticket_id', 'timestamp', 'complaint_text', 'department_id', 'district', 'neighborhood', 'latitude', 'longitude', 'platform', 'status', 'actual_category']

✓ Sample data:
   ticket_id  department_id  actual_category  \
0          1              1  su_kanalizasyon   
1          2              1  su_kanalizasyon   
2          3              1  su_kanalizasyon   
3          4              1  su_kanalizasyon   
4          5              1  su_kanalizasyon   

                                      complaint_text  
0  Günaydın, apartmanımızda 3 gündür su basıncı ç...  
1  Merhabalar, Kennedy Sokağında sabahtan beri su...  
2  Günaydın, apartmanımızda 3 gündür su basıncı ç...  
3  İyi günler, su faturamız anormal derecede yüks...  
4  Merhaba, sokağımızda 2 gündür kanalizasyon taş...  


---
## 🔤 SECTION 4: KEYWORD-BASED CLASSIFICATION
Define category-specific keywords and implement conservative classification with entity name boosting.

In [14]:
# ====================
# STEP 1: Define Category Keywords
# ====================

print("\n" + "="*80)
print("KEYWORD-BASED CLASSIFICATION - IMPLEMENTATION")
print("="*80)

# Comprehensive keyword dictionary
CATEGORY_KEYWORDS = {
    'ulasim_trafik': {
        'strong': ['iett', 'ukome', 'metrobüs'],  # Entity names - very reliable
        'medium': ['otobüs', 'metro', 'tramvay', 'hat', 'sefer', 'durak'],
        'weak': ['otopark', 'trafik', 'yol']  # Can appear in other contexts
    },
    'sikayet_takip': {
        'strong': ['alo 153', '153', 'beyaz masa', 'cimer'],
        'medium': ['başvuru', 'takip', 'şikayet', 'çağrı', 'randevu'],
        'weak': ['aradım', 'aramadı']
    },
    'su_kanalizasyon': {
        'strong': ['iski', 'İski'],
        'medium': ['kanalizasyon', 'lağım', 'su sayacı', 'musluk'],
        'weak': ['su', 'sızıntı', 'boru']
    },
    'atik_yonetimi_temizlik': {
        'strong': [],
        'medium': ['çöp', 'konteyner', 'geri dönüşüm', 'atık toplama'],
        'weak': ['temizlik', 'moloz', 'süpür']
    },
    'yol_altyapi': {
        'strong': [],
        'medium': ['asfalt', 'çukur', 'kaldırım', 'bordür', 'kavşak'],
        'weak': ['yol', 'bisiklet yolu']
    },
    'aydinlatma': {
        'strong': ['sokak lambası'],
        'medium': ['lamba', 'aydınlatma', 'ampul', 'led', 'direk'],
        'weak': ['ışık', 'karanlık']
    },
    'yesil_alan_bahce': {
        'strong': ['oyun alanı', 'çocuk parkı'],
        'medium': ['park', 'bahçe', 'yeşil alan', 'salıncak', 'spor aleti'],
        'weak': ['ağaç', 'çocuk']
    },
    'sosyal_yardim': {
        'strong': ['sedep', 'mavi kart', '65+'],
        'medium': ['gıda yardımı', 'kömür yardımı', 'engelli kartı'],
        'weak': ['yardım', 'başvuru']
    },
    'fatura_odeme': {
        'strong': ['emlak vergisi', 'çevre temizlik vergisi'],
        'medium': ['fatura', 'borç', 'tahsilat', 'taksit'],
        'weak': ['ödeme', 'vergi']
    },
    'ruhsat_imar': {
        'strong': ['yapı ruhsatı', 'işyeri ruhsatı', 'imar durumu'],
        'medium': ['ruhsat', 'imar', 'belge', 'tasdik'],
        'weak': ['evrak', 'başvuru']
    },
    'dijital_sistem': {
        'strong': ['e-belediye', 'istanbul senin', 'ibb web'],
        'medium': ['web sitesi', 'mobil uygulama', 'qr kod', 'sms doğrulama'],
        'weak': ['sistem', 'uygulama', 'site']
    }
}

# Count keywords
total_keywords = 0
for cat, kw_dict in CATEGORY_KEYWORDS.items():
    count = sum(len(v) for v in kw_dict.values())
    total_keywords += count
    print(f"  {cat}: {count} keywords")

print(f"\nTotal: {total_keywords} keywords defined across {len(CATEGORY_KEYWORDS)} categories")
print("="*80)


KEYWORD-BASED CLASSIFICATION - IMPLEMENTATION
  ulasim_trafik: 12 keywords
  sikayet_takip: 11 keywords
  su_kanalizasyon: 9 keywords
  atik_yonetimi_temizlik: 7 keywords
  yol_altyapi: 7 keywords
  aydinlatma: 8 keywords
  yesil_alan_bahce: 9 keywords
  sosyal_yardim: 8 keywords
  fatura_odeme: 8 keywords
  ruhsat_imar: 9 keywords
  dijital_sistem: 10 keywords

Total: 98 keywords defined across 11 categories


In [15]:
# ====================
# CONSERVATIVE CLASSIFICATION WITH ENTITY NAME KEYWORDS
# ====================

def classify_complaint_conservative(complaint_text, model, template_embeddings, top_k=3):

    # PART 1: Semantic Similarity
    complaint_embedding = model.encode([complaint_text])[0]
    similarities = cosine_similarity([complaint_embedding], template_embeddings)[0]
    
    category_scores = {}
    for idx in range(len(similarities)):
        category = template_categories[idx]
        score = similarities[idx]
        if category not in category_scores:
            category_scores[category] = []
        category_scores[category].append(score)
    
    category_avg_scores = {
        cat: np.mean(scores) for cat, scores in category_scores.items()
    }
    
    # PART 2: CONSERVATIVE KEYWORD BOOSTING - Only strong keywords (entity names)
    text_lower = complaint_text.lower()
    STRONG_BOOST = 0.25  # Higher boost since we're very confident
    
    for category, kw_dict in CATEGORY_KEYWORDS.items():
        if category not in category_avg_scores:
            continue
        
        # Only use strong keywords (entity names)
        for keyword in kw_dict.get('strong', []):
            if keyword in text_lower:
                category_avg_scores[category] += STRONG_BOOST
                break  # One entity match is enough
    
    # Get top category and top-k
    top_category = max(category_avg_scores.items(), key=lambda x: x[1])
    sorted_categories = sorted(
        category_avg_scores.items(),
        key=lambda x: x[1],
        reverse=True
    )[:top_k]
    
    return {
        'category': top_category[0],
        'confidence': float(top_category[1]),
        'top_k': [
            {'category': cat, 'score': float(score)}
            for cat, score in sorted_categories
        ]
    }

---
## 🎯 FINAL CLASSIFICATION - Apply to All Complaints
Run conservative classification on all complaints and route low-confidence cases to manual review.

In [16]:
# ====================
# RUN FINAL CLASSIFICATION ON ALL COMPLAINTS
# ====================

print("\n" + "="*80)
print("FINAL CLASSIFICATION - CONSERVATIVE APPROACH WITH 'DİĞER' ROUTING")
print("="*80)

# Configuration
CONFIDENCE_THRESHOLD = 0.34  # Optimal threshold for ~14% manual review
print(f"\nConfiguration:")
print(f"  Approach: Conservative (entity name keywords only)")
print(f"  Confidence threshold: {CONFIDENCE_THRESHOLD}")

# Classify all complaints
print(f"\n\nClassifying {len(complaints)} complaints...")

conservative_results = []
for idx, row in complaints.iterrows():
    if idx % 100 == 0:
        print(f"  Progress: {idx+1}/{len(complaints)}...")
    
    result = classify_complaint_conservative(
        row['complaint_text'], 
        model, 
        template_embeddings
    )
    conservative_results.append(result)

# Add results to dataframe
complaints['predicted_conservative'] = [r['category'] for r in conservative_results]
complaints['conservative_confidence'] = [r['confidence'] for r in conservative_results]

# Apply "diğer" routing based on confidence threshold
complaints['final_category'] = complaints.apply(
    lambda row: 'diğer' if row['conservative_confidence'] < CONFIDENCE_THRESHOLD else row['predicted_conservative'],
    axis=1
)

print("\n" + "="*80)
print("CLASSIFICATION COMPLETE")
print("="*80)

# Calculate metrics
total = len(complaints)
auto_classified = (complaints['final_category'] != 'diğer').sum()
manual_review = (complaints['final_category'] == 'diğer').sum()

# Accuracy on auto-classified
high_conf = complaints[complaints['final_category'] != 'diğer']
if len(high_conf) > 0:
    high_conf_correct = (high_conf['predicted_conservative'] == high_conf['actual_category']).sum()
    high_conf_accuracy = high_conf_correct / len(high_conf) * 100
else:
    high_conf_accuracy = 0
    high_conf_correct = 0

# Overall stats
print(f"\n📊 System Performance:")
print(f"   Total complaints: {total}")
print(f"   ✓ Auto-classified: {auto_classified} ({auto_classified/total*100:.1f}%)")
print(f"     └─ Accuracy: {high_conf_accuracy:.1f}% ({high_conf_correct}/{len(high_conf)})")
print(f"   ⚠ Manual review (diğer): {manual_review} ({manual_review/total*100:.1f}%)")

# Confidence statistics
print(f"\n📈 Confidence Scores:")
print(f"   Mean: {complaints['conservative_confidence'].mean():.3f}")
print(f"   Median: {complaints['conservative_confidence'].median():.3f}")
print(f"   Auto-classified range: {complaints[complaints['final_category'] != 'diğer']['conservative_confidence'].min():.3f} - {complaints[complaints['final_category'] != 'diğer']['conservative_confidence'].max():.3f}")
if manual_review > 0:
    print(f"   Manual review range: {complaints[complaints['final_category'] == 'diğer']['conservative_confidence'].min():.3f} - {complaints[complaints['final_category'] == 'diğer']['conservative_confidence'].max():.3f}")

# Categories distribution in auto-classified
print(f"\n📁 Auto-Classified Distribution:")
auto_dist = complaints[complaints['final_category'] != 'diğer']['final_category'].value_counts()
for cat, count in auto_dist.items():
    pct = count / auto_classified * 100
    print(f"   {cat}: {count} ({pct:.1f}%)")

# What's in diğer?
if manual_review > 0:
    print(f"\n🔍 Manual Review ('diğer') - Actual Category Distribution:")
    diger_dist = complaints[complaints['final_category'] == 'diğer']['actual_category'].value_counts().head(5)
    for cat, count in diger_dist.items():
        pct = count / manual_review * 100
        print(f"   {cat}: {count} ({pct:.1f}%)")

print("\n" + "="*80)
print("✓ Classification ready for database insertion")
print("  Column: 'final_category' (includes 'diğer' routing)")
print("="*80)


FINAL CLASSIFICATION - CONSERVATIVE APPROACH WITH 'DİĞER' ROUTING

Configuration:
  Approach: Conservative (entity name keywords only)
  Confidence threshold: 0.34


Classifying 1000 complaints...
  Progress: 1/1000...
  Progress: 101/1000...
  Progress: 201/1000...
  Progress: 301/1000...
  Progress: 401/1000...
  Progress: 501/1000...
  Progress: 601/1000...
  Progress: 701/1000...
  Progress: 801/1000...
  Progress: 901/1000...

CLASSIFICATION COMPLETE

📊 System Performance:
   Total complaints: 1000
   ✓ Auto-classified: 861 (86.1%)
     └─ Accuracy: 83.2% (716/861)
   ⚠ Manual review (diğer): 139 (13.9%)

📈 Confidence Scores:
   Mean: 0.472
   Median: 0.472
   Auto-classified range: 0.343 - 0.768
   Manual review range: 0.227 - 0.340

📁 Auto-Classified Distribution:
   atik_yonetimi_temizlik: 121 (14.1%)
   aydinlatma: 116 (13.5%)
   sikayet_takip: 115 (13.4%)
   ruhsat_imar: 98 (11.4%)
   dijital_sistem: 85 (9.9%)
   su_kanalizasyon: 75 (8.7%)
   yol_altyapi: 66 (7.7%)
   yesil_

---
## 🗄️ SECTION 4: DATABASE OPERATIONS
Load classified complaints into PostgreSQL with pgvector for semantic search.

In [17]:
# Install required packages for database
!pip install -q psycopg2-binary

In [18]:
# Database setup - Start PostgreSQL Docker container
import subprocess
import time

print("="*80)
print("STARTING POSTGRESQL DATABASE")
print("="*80)

# Create docker-compose.yml if it doesn't exist
docker_compose = """version: '3.8'
services:
  postgres:
    image: pgvector/pgvector:pg16
    container_name: istanbuilders_db
    environment:
      POSTGRES_DB: complaints_db
      POSTGRES_USER: istanbuilders
      POSTGRES_PASSWORD: istanbuilders123
    ports:
      - "5432:5432"
    volumes:
      - postgres_data:/var/lib/postgresql/data

volumes:
  postgres_data:
"""

with open('docker-compose.yml', 'w') as f:
    f.write(docker_compose)

print("\n✓ Created docker-compose.yml")

# Start Docker container
print("\nStarting PostgreSQL container...")
result = subprocess.run(['docker-compose', 'up', '-d'], capture_output=True, text=True)

if result.returncode == 0:
    print("✓ PostgreSQL container started successfully!")
    print("\nWaiting for database to be ready...")
    time.sleep(5)
    print("✓ Database is ready!")
else:
    print(f"✗ Failed to start container: {result.stderr}")
    print("\nMake sure Docker is installed and running:")
    print("  https://www.docker.com/products/docker-desktop")

print("\n" + "="*80)

STARTING POSTGRESQL DATABASE

✓ Created docker-compose.yml

Starting PostgreSQL container...
✓ PostgreSQL container started successfully!

Waiting for database to be ready...
✓ Database is ready!



In [19]:
# Connect to database and create schema
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'complaints_db',
    'user': 'istanbuilders',
    'password': 'istanbuilders123'
}

print("\n" + "="*80)
print("CREATING DATABASE SCHEMA")
print("="*80)

try:
    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()
    
    # Enable pgvector extension
    print("\nEnabling pgvector extension...")
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    print("✓ pgvector enabled")
    
    # Drop existing tables
    print("\nDropping existing tables...")
    cur.execute("DROP TABLE IF EXISTS complaint_embeddings CASCADE;")
    cur.execute("DROP TABLE IF EXISTS complaints CASCADE;")
    cur.execute("DROP TABLE IF EXISTS departments CASCADE;")
    print("✓ Dropped old tables")
    
    # Create tables
    print("\nCreating tables...")
    
    # Departments table
    cur.execute("""
        CREATE TABLE departments (
            department_id INTEGER PRIMARY KEY,
            department_name VARCHAR(255) NOT NULL,
            category_name VARCHAR(100) NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """)
    print("✓ Created departments table")
    
    # Complaints table
    cur.execute("""
        CREATE TABLE complaints (
            complaint_id INTEGER PRIMARY KEY,
            timestamp TIMESTAMP,
            complaint_text TEXT NOT NULL,
            department_id INTEGER REFERENCES departments(department_id),
            district VARCHAR(100),
            neighborhood VARCHAR(100),
            latitude FLOAT,
            longitude FLOAT,
            platform VARCHAR(100),
            predicted_category VARCHAR(100),
            prediction_confidence FLOAT,
            final_category VARCHAR(100),
            status VARCHAR(50),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """)
    print("✓ Created complaints table")
    
    # Embeddings table
    cur.execute("""
        CREATE TABLE complaint_embeddings (
            complaint_id INTEGER PRIMARY KEY REFERENCES complaints(complaint_id),
            embedding vector(768),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """)
    print("✓ Created complaint_embeddings table")
    
    # Create indexes
    print("\nCreating indexes...")
    cur.execute("CREATE INDEX idx_complaints_category ON complaints(predicted_category);")
    cur.execute("CREATE INDEX idx_complaints_confidence ON complaints(prediction_confidence);")
    cur.execute("CREATE INDEX idx_complaints_department ON complaints(department_id);")
    print("✓ Created indexes")
    
    cur.close()
    conn.close()
    
    print("\n✓ Database schema created successfully!")
    print("="*80)
    
except Exception as e:
    print(f"\n✗ Error: {e}")
    print("\nMake sure PostgreSQL container is running:")
    print("  docker-compose up -d")


CREATING DATABASE SCHEMA

Enabling pgvector extension...
✓ pgvector enabled

Dropping existing tables...
✓ Dropped old tables

Creating tables...
✓ Created departments table
✓ Created complaints table
✓ Created complaint_embeddings table

Creating indexes...
✓ Created indexes

✓ Database schema created successfully!


In [20]:
# Insert data into database
print("\n" + "="*80)
print("INSERTING DATA INTO DATABASE")
print("="*80)

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Insert departments
    print("\nInserting departments...")
    for _, row in departments.iterrows():
        cur.execute("""
            INSERT INTO departments (department_id, department_name, category_name)
            VALUES (%s, %s, %s)
            ON CONFLICT (department_id) DO UPDATE SET department_name = EXCLUDED.department_name, category_name = EXCLUDED.category_name;
        """, (int(row['department_id']), row['department_name'], row['category_name']))
    print(f"✓ Inserted {len(departments)} departments")
    
    # Insert complaints
    print("\nInserting complaints...")
    for _, row in complaints.iterrows():
        cur.execute("""
            INSERT INTO complaints (
                complaint_id, timestamp, complaint_text, department_id,
                district, neighborhood, latitude, longitude,
                platform, predicted_category, prediction_confidence, final_category, status
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (complaint_id) DO UPDATE SET
                predicted_category = EXCLUDED.predicted_category,
                prediction_confidence = EXCLUDED.prediction_confidence,
                final_category = EXCLUDED.final_category,
                status = EXCLUDED.status;
        """, (
            int(row['ticket_id']),
            row.get('timestamp', None),
            row['complaint_text'],
            int(row['department_id']),
            row.get('district', 'Kadıköy'),
            row.get('neighborhood', 'Moda'),
            row.get('latitude', 40.9850 + np.random.uniform(-0.1, 0.1)),
            row.get('longitude', 29.0360 + np.random.uniform(-0.1, 0.1)),
            row.get('platform', 'İstanbul Senin'),
            row['predicted_conservative'],
            float(row['conservative_confidence']),
            row['final_category'],
            row.get('status', 'basvuru_alindi')
        ))
    
    print(f"✓ Inserted {len(complaints)} complaints")
    
    # Insert embeddings
    print("\nComputing and inserting embeddings...")
    batch_size = 100
    for i in range(0, len(complaints), batch_size):
        batch = complaints.iloc[i:i+batch_size]
        texts = batch['complaint_text'].tolist()
        embeddings = model.encode(texts, show_progress_bar=False)
        
        for j, (_, row) in enumerate(batch.iterrows()):
            embedding_list = embeddings[j].tolist()
            cur.execute("""
                INSERT INTO complaint_embeddings (complaint_id, embedding)
                VALUES (%s, %s)
                ON CONFLICT (complaint_id) DO UPDATE SET embedding = EXCLUDED.embedding;
            """, (int(row['ticket_id']), embedding_list))
        
        print(f"  Progress: {min(i+batch_size, len(complaints))}/{len(complaints)} embeddings inserted")
    
    print(f"✓ Inserted {len(complaints)} embeddings")
    
    conn.commit()
    cur.close()
    conn.close()
    
    print("\n✓ All data inserted successfully!")
    print("="*80)
    
except Exception as e:
    print(f"\n✗ Error: {e}")
    if 'conn' in locals():
        conn.rollback()


INSERTING DATA INTO DATABASE

Inserting departments...
✓ Inserted 11 departments

Inserting complaints...
✓ Inserted 1000 complaints

Computing and inserting embeddings...
  Progress: 100/1000 embeddings inserted
  Progress: 200/1000 embeddings inserted
  Progress: 300/1000 embeddings inserted
  Progress: 400/1000 embeddings inserted
  Progress: 500/1000 embeddings inserted
  Progress: 600/1000 embeddings inserted
  Progress: 700/1000 embeddings inserted
  Progress: 800/1000 embeddings inserted
  Progress: 900/1000 embeddings inserted
  Progress: 1000/1000 embeddings inserted
✓ Inserted 1000 embeddings

✓ All data inserted successfully!


In [21]:
# Create pk_ilce table with Istanbul districts
print("Creating pk_ilce table with Istanbul districts...")

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Create table
    cur.execute("""
        DROP TABLE IF EXISTS pk_ilce;
        CREATE TABLE pk_ilce (
            ilce_id SERIAL PRIMARY KEY,
            il_id INTEGER NOT NULL,
            ilce_adi VARCHAR(255) NOT NULL,
            lat DOUBLE PRECISION,
            lng DOUBLE PRECISION,
            northeast_lat DOUBLE PRECISION,
            northeast_lng DOUBLE PRECISION,
            southwest_lat DOUBLE PRECISION,
            southwest_lng DOUBLE PRECISION
        );
        CREATE INDEX idx_il_id ON pk_ilce(il_id);
        CREATE INDEX idx_ilce_adi ON pk_ilce(ilce_adi);
    """)
    
    # Insert Istanbul districts from SQL file
    # Data extracted from il_ilce_enlem_boylam.sql for il_id = 34
    istanbul_districts = [
        (34, 'ADALAR', 40.87637720, 29.09544400, 40.91462700, 29.14819400, 40.83737600, 29.03911400),
        (34, 'ARNAVUTKÖY', 41.20000000, 28.73333300, 41.20839490, 28.74934040, 41.19160400, 28.71732560),
        (34, 'ATAŞEHİR', 40.98333330, 29.11666670, 41.00952500, 29.21224000, 40.95902900, 29.07224400),
        (34, 'AVCILAR', 41.01534790, 28.73146180, 41.10611300, 28.75669200, 40.96910000, 28.68197900),
        (34, 'BAĞCILAR', 41.04555560, 28.84055560, 41.07221400, 28.87269000, 41.01756500, 28.80796700),
        (34, 'BAHÇELİEVLER', 40.99750000, 28.85055560, 41.02785000, 28.88474600, 40.99166000, 28.80590300),
        (34, 'BAKIRKÖY', 40.96815500, 28.82280000, 41.00714200, 28.89662200, 40.95450800, 28.77347200),
        (34, 'BAŞAKŞEHİR', 41.07789500, 28.81255100, 41.14944600, 28.82392400, 41.05784400, 28.63913000),
        (34, 'BAYRAMPAŞA', 41.04815030, 28.90045530, 41.07710100, 28.92398000, 41.02585200, 28.87844800),
        (34, 'BEŞİKTAŞ', 41.06861600, 29.02853550, 41.10645200, 29.05509800, 41.03676200, 28.99182600),
        (34, 'BEYKOZ', 41.13271900, 29.10569000, 41.14112250, 29.12169740, 41.12431440, 29.08968260),
        (34, 'BEYLİKDÜZÜ', 40.99103810, 28.64981440, 41.01863100, 28.69839000, 40.95730500, 28.59555400),
        (34, 'BEYOĞLU', 41.03828640, 28.97033040, 41.06507100, 28.99575500, 41.02102500, 28.93900400),
        (34, 'BÜYÜKÇEKMECE', 41.03413300, 28.59000300, 41.12965400, 28.63482100, 40.98457600, 28.39098300),
        (34, 'ÇATALCA', 41.14823900, 28.46773000, 41.15664050, 28.48373740, 41.13983640, 28.45172260),
        (34, 'ÇEKMEKÖY', 41.10423500, 29.31772720, 41.13267000, 29.40112500, 41.01297900, 29.13837900),
        (34, 'ESENLER', 41.07941330, 28.85385450, 41.13579700, 28.90062600, 41.01727500, 28.81347900),
        (34, 'ESENYURT', 41.03428060, 28.68011940, 41.09853900, 28.70190000, 40.99907100, 28.62052500),
        (34, 'EYÜP', 41.18715980, 28.88298160, 41.30653700, 28.97628300, 41.02433900, 28.77050700),
        (34, 'FATİH', 41.01666670, 28.93333330, 41.04084100, 28.98807200, 40.98763700, 28.91903700),
        (34, 'GAZİOSMANPAŞA', 41.07594770, 28.90045530, 41.09674100, 28.93919500, 41.05159200, 28.87438900),
        (34, 'GÜNGÖREN', 41.01666670, 28.88333330, 41.03847300, 28.90090500, 41.00138600, 28.85849300),
        (34, 'KADIKÖY', 40.98014100, 29.08227000, 41.01226800, 29.11125700, 40.94999800, 29.01484900),
        (34, 'KAĞITHANE', 41.07100000, 28.97000000, 41.11295900, 29.00810700, 41.05290800, 28.94937200),
        (34, 'KARTAL', 40.89965100, 29.19364900, 40.96492500, 29.25519900, 40.87812900, 29.13828900),
        (34, 'KÜÇÜKÇEKMECE', 41.00865800, 28.77534200, 41.06410300, 28.82170000, 40.97727600, 28.73902300),
        (34, 'MALTEPE', 40.94904700, 29.17410900, 40.98123500, 29.21627700, 40.90663700, 29.09481000),
        (34, 'PENDİK', 40.87932600, 29.25813500, 40.88776190, 29.27414240, 40.87088900, 29.24212760),
        (34, 'SANCAKTEPE', 41.02870280, 29.29018290, 41.05272200, 29.36434400, 40.94604600, 29.18291400),
        (34, 'SARIYER', 41.16632800, 29.04995000, 41.17472720, 29.06595740, 41.15792770, 29.03394260),
        (34, 'SİLİVRİ', 41.08015800, 28.26829000, 41.08856830, 28.28429740, 41.07174670, 28.25228260),
        (34, 'SULTANBEYLİ', 40.96111230, 29.26694380, 41.00489600, 29.31198900, 40.93176800, 29.24470600),
        (34, 'SULTANGAZİ', 41.12557940, 28.87133140, 41.15973500, 28.92242200, 41.08154100, 28.82449400),
        (34, 'ŞİLE', 41.17638890, 29.61277780, 41.18225200, 29.65835000, 41.15274200, 29.56819900),
        (34, 'ŞİŞLİ', 41.06000000, 28.98700000, 41.14928900, 29.02592200, 41.03919600, 28.95866800),
        (34, 'TUZLA', 40.84200000, 29.29500000, 40.98064800, 29.45652300, 40.80268900, 29.25462100),
        (34, 'ÜMRANİYE', 41.03030000, 29.10650000, 41.06554500, 29.19719000, 40.98896500, 29.08031300),
        (34, 'ÜSKÜDAR', 41.03223600, 29.03193800, 41.07799700, 29.09171800, 40.99250000, 29.00636200),
        (34, 'ZEYTİNBURNU', 40.99063500, 28.89614000, 41.03247600, 28.92899600, 40.97784400, 28.88944400),
    ]
    
    # Insert data
    for district in istanbul_districts:
        cur.execute("""
            INSERT INTO pk_ilce (il_id, ilce_adi, lat, lng, northeast_lat, northeast_lng, southwest_lat, southwest_lng)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, district)
    
    conn.commit()
    cur.close()
    conn.close()
    
    print(f"✓ pk_ilce table created with {len(istanbul_districts)} Istanbul districts")
    
except Exception as e:
    print(f"✗ Error creating pk_ilce table: {e}")
    if 'conn' in locals():
        conn.rollback()
        conn.close()

Creating pk_ilce table with Istanbul districts...
✓ pk_ilce table created with 39 Istanbul districts


In [23]:
# Create pk_mahalle table for Istanbul neighborhoods (schema: ilce_adi, mahalle_adi, lat, lng)
print("Creating pk_mahalle table for Istanbul neighborhoods...")

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("""
        DROP TABLE IF EXISTS pk_mahalle;
        CREATE TABLE pk_mahalle (
            mahalle_id SERIAL PRIMARY KEY,
            ilce_adi VARCHAR(255) NOT NULL,
            mahalle_adi VARCHAR(255) NOT NULL,
            lat DOUBLE PRECISION,
            lng DOUBLE PRECISION
        );
        CREATE INDEX idx_pk_mahalle_ilce_adi ON pk_mahalle(ilce_adi);
        CREATE INDEX idx_pk_mahalle_mahalle_adi ON pk_mahalle(mahalle_adi);
    """)
    conn.commit()
    cur.close()
    conn.close()
    print("✓ pk_mahalle table created.")
except Exception as e:
    print(f"✗ Error creating pk_mahalle table: {e}")
    if 'conn' in locals():
        conn.rollback()
        conn.close()

Creating pk_mahalle table for Istanbul neighborhoods...
✓ pk_mahalle table created.


In [ ]:
# Import all Istanbul neighborhoods from neighborhoods.sql into pk_mahalle
# FIXED: Use 'town' column as district name (converted to uppercase), 'name' column as neighborhood name
import psycopg2
import re

print("Importing Istanbul neighborhoods into pk_mahalle table (FIXED VERSION)...")

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    # Clear existing data first
    cur.execute("DELETE FROM pk_mahalle")
    print("✓ Cleared existing pk_mahalle data")

    # Updated pattern to capture town column (district)
    pattern = re.compile(r"'([0-9a-f]+)','(İstanbul|Istanbul)','([^']+)','([^']+)','([^']+)',([0-9]+)")
    istanbul_rows = []

    with open("enlem_boylam_data/neighborhoods.sql", encoding="utf-8") as f:
        for line in f:
            if "'İstanbul'" in line or "'Istanbul'" in line:
                match = pattern.search(line)
                if match:
                    # town = district name (convert to uppercase), name = neighborhood name
                    ilce_adi = match.group(5).strip().upper()  # town column = actual district, convert to uppercase
                    mahalle_adi = match.group(4).strip()  # name column = neighborhood
                    istanbul_rows.append((ilce_adi, mahalle_adi))

    print(f"Found {len(istanbul_rows)} Istanbul neighborhoods.")

    # Remove duplicates
    istanbul_rows = list(set(istanbul_rows))
    print(f"Unique neighborhoods: {len(istanbul_rows)}")

    # Insert into pk_mahalle (lat/lng as NULL for now)
    cur.executemany(
        """
        INSERT INTO pk_mahalle (ilce_adi, mahalle_adi, lat, lng)
        VALUES (%s, %s, NULL, NULL)
        """,
        istanbul_rows
    )

    conn.commit()
    cur.close()
    conn.close()
    print("✓ All Istanbul neighborhoods imported into pk_mahalle with correct district mapping.")

except Exception as e:
    print(f"✗ Error importing neighborhoods: {e}")
    if 'conn' in locals():
        conn.rollback()
        conn.close()

Importing Istanbul neighborhoods into pk_mahalle table...
Found 971 Istanbul neighborhoods.
Unique neighborhoods: 940
✓ All Istanbul neighborhoods imported into pk_mahalle.


In [30]:
# List all tables in the current PostgreSQL database
import psycopg2

print("Listing all tables in the database...")
try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """)
    tables = cur.fetchall()
    print("Tables in the database:")
    for t in tables:
        print("  -", t[0])
    cur.close()
    conn.close()
except Exception as e:
    print(f"✗ Error listing tables: {e}")

Listing all tables in the database...
Tables in the database:
  - complaint_embeddings
  - complaints
  - departments
  - pk_ilce
  - pk_mahalle


In [31]:
# Debug: Check pk_mahalle data for a district
district_name = "Kadıköy"  # Change as needed

import psycopg2

print(f"Checking pk_mahalle data for {district_name}:")
try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    # Check if table exists and has data
    cur.execute("SELECT COUNT(*) FROM pk_mahalle")
    total_count = cur.fetchone()[0]
    print(f"Total rows in pk_mahalle: {total_count}")

    # Check distinct districts
    cur.execute("SELECT DISTINCT ilce_adi FROM pk_mahalle ORDER BY ilce_adi")
    districts = [row[0] for row in cur.fetchall()]
    print(f"Distinct districts in pk_mahalle: {districts}")

    # Check neighborhoods for specific district
    cur.execute("""
        SELECT mahalle_adi
        FROM pk_mahalle
        WHERE ilce_adi = %s
        ORDER BY mahalle_adi
    """, (district_name,))
    neighborhoods = cur.fetchall()
    print(f"Neighborhoods for {district_name}: {[row[0] for row in neighborhoods]}")
    print(f"Count: {len(neighborhoods)}")

    cur.close()
    conn.close()
except Exception as e:
    print(f"✗ Error: {e}")

Checking pk_mahalle data for Kadıköy:
Total rows in pk_mahalle: 2820
Distinct districts in pk_mahalle: ['Abbasağa', 'Acıbadem', 'Ağva', 'Ahmetli', 'Akatlar', 'Akfırat', 'Akıncılar', 'Aksaray', 'Alemdağ', 'Alibeyköy', 'Altınşehir', 'Altıntepsi', 'Altunizade', 'Ambarlı', 'Anadoluhisarı', 'Arapcami', 'Arnavutköy', 'Aşağıdudullu', 'Atakent', 'Ataköy', 'Atalar', 'Atatürk', 'Atışalanı', 'Aydınevler', 'Aydınlı', 'Bahçeköy', 'Bahçelievler', 'Bahçeşehir', 'Baklalı', 'Balat', 'Bayazıt', 'Bebek', 'Beylerbeyi', 'Boğazköy', 'Bolluca', 'Bostancı', 'Bozkurt', 'Bulgurlu', 'Burgazada', 'Büyükada', 'Büyükbakkalköy', 'Büyükçavuşlu', 'Büyükdere', 'Caddebostan', 'Caferağa', 'Çağlayan', 'Çakmak', 'Çamçeşme', 'Çanta', 'Çarşı', 'Çavuşbaşı', 'Celaliye', 'Çeliktepe', 'Çengelköy', 'Cevizli', 'Cevizlik', 'Cihangir', 'Çınardere', 'Çırçır', 'Çırpıcı', 'Çubuklu', 'Cumhuriyet', 'Değirmenköy', 'Duatepe', 'Eğitim', 'Eminönü', 'Emirgan', 'Erenköy', 'Esenşehir', 'Esentepe', 'Esenyalı', 'Etiler', 'Eyüp', 'Eyüpsultan', 'Fe

In [ ]:
# Compare district names between pk_ilce and pk_mahalle
import psycopg2

print("Comparing district names between pk_ilce and pk_mahalle:")
try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    # Get districts from pk_ilce
    cur.execute("SELECT DISTINCT ilce_adi FROM pk_ilce ORDER BY ilce_adi")
    ilce_districts = [row[0] for row in cur.fetchall()]

    # Get districts from pk_mahalle
    cur.execute("SELECT DISTINCT ilce_adi FROM pk_mahalle ORDER BY ilce_adi")
    mahalle_districts = [row[0] for row in cur.fetchall()]

    print(f"Districts in pk_ilce ({len(ilce_districts)}): {ilce_districts[:10]}...")  # Show first 10
    print(f"Districts in pk_mahalle ({len(mahalle_districts)}): {mahalle_districts[:10]}...")  # Show first 10

    # Find matches and differences
    matches = set(ilce_districts) & set(mahalle_districts)
    in_ilce_not_mahalle = set(ilce_districts) - set(mahalle_districts)
    in_mahalle_not_ilce = set(mahalle_districts) - set(ilce_districts)

    print(f"\nMatching districts: {len(matches)}")
    print(f"Districts in pk_ilce but not in pk_mahalle: {sorted(list(in_ilce_not_mahalle))}")
    print(f"Districts in pk_mahalle but not in pk_ilce: {sorted(list(in_mahalle_not_ilce))[:10]}...")  # Show first 10

    cur.close()
    conn.close()
except Exception as e:
    print(f"✗ Error: {e}")

Comparing district names between pk_ilce and pk_mahalle:
Districts in pk_ilce: ['ADALAR', 'ARNAVUTKÖY', 'ATAŞEHİR', 'AVCILAR', 'BAĞCILAR', 'BAHÇELİEVLER', 'BAKIRKÖY', 'BAŞAKŞEHİR', 'BAYRAMPAŞA', 'BEŞİKTAŞ', 'BEYKOZ', 'BEYLİKDÜZÜ', 'BEYOĞLU', 'BÜYÜKÇEKMECE', 'ÇATALCA', 'ÇEKMEKÖY', 'ESENLER', 'ESENYURT', 'EYÜP', 'FATİH', 'GAZİOSMANPAŞA', 'GÜNGÖREN', 'KADIKÖY', 'KAĞITHANE', 'KARTAL', 'KÜÇÜKÇEKMECE', 'MALTEPE', 'PENDİK', 'SANCAKTEPE', 'SARIYER', 'ŞİLE', 'SİLİVRİ', 'ŞİŞLİ', 'SULTANBEYLİ', 'SULTANGAZİ', 'TUZLA', 'ÜMRANİYE', 'ÜSKÜDAR', 'ZEYTİNBURNU']
Districts in pk_mahalle: ['Abbasağa', 'Acıbadem', 'Ağva', 'Ahmetli', 'Akatlar', 'Akfırat', 'Akıncılar', 'Aksaray', 'Alemdağ', 'Alibeyköy', 'Altınşehir', 'Altıntepsi', 'Altunizade', 'Ambarlı', 'Anadoluhisarı', 'Arapcami', 'Arnavutköy', 'Aşağıdudullu', 'Atakent', 'Ataköy', 'Atalar', 'Atatürk', 'Atışalanı', 'Aydınevler', 'Aydınlı', 'Bahçeköy', 'Bahçelievler', 'Bahçeşehir', 'Baklalı', 'Balat', 'Bayazıt', 'Bebek', 'Beylerbeyi', 'Boğazköy', 'Bolluca', 

---
## 🎨 SECTION 5: DEMO UI
Simple Streamlit-based web interface for testing the classification system.

In [26]:
# Demo UI Application
print("✓ Demo app created: demo_app.py")
print()
print("To run the İBB Çözüm Merkezi demo:")
print("  streamlit run demo_app.py")
print()
print("The app will open at http://localhost:8501")
print()
print("Features:")
print("  • İBB Çözüm Merkezi branding")
print("  • All 39 Istanbul districts (İlçe) from database")
print("  • Real coordinates from pk_ilce table")
print("  • Auto-generated complaint_id (incrementing)")
print("  • Auto-generated timestamp")
print("  • Real-time classification")
print("  • Clean user-focused interface (no stats or recent submissions)")
print("  • Save to database with district coordinates")


✓ Demo app created: demo_app.py

To run the İBB Çözüm Merkezi demo:
  streamlit run demo_app.py

The app will open at http://localhost:8501

Features:
  • İBB Çözüm Merkezi branding
  • All 39 Istanbul districts (İlçe) from database
  • Real coordinates from pk_ilce table
  • Auto-generated complaint_id (incrementing)
  • Auto-generated timestamp
  • Real-time classification
  • Clean user-focused interface (no stats or recent submissions)
  • Save to database with district coordinates
